In [12]:
# 요기요 크롤링
from selenium import webdriver
import time  # 지정된 시간동안 파이썬에 렉을 거는 기능 
from selenium.webdriver.support.ui import WebDriverWait # Chrome이 웹페이지 로딩을 완료할 때까지 최대 n초간 대기하는 기능
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By ## 원하는 부분을 추출할 수 있는 방식을 지정하는 객체
from pandas import DataFrame



In [82]:
# 검색할 웺사이트 
yogiyo = 'https://www.yogiyo.co.kr/mobile/#/'

# 검색할 지역 
location = '서울특별시 서초구 방배동 836-10 삼경빌딩'

In [95]:
# url입력
driver = webdriver.Chrome() 
driver.implicitly_wait(5) # 크롬 브라우저가 준비될 때까지 최대 5초씩 대기해라 
driver.get(yogiyo) #요기요 페이지로 이동 
time.sleep(1) # 페이지가 다 뜰때까지 기다려라 

In [96]:
# 검색창 선택
xpath = '''//*[@id="search"]/div/form/input''' # 절대경로 
element = driver.find_element(By.XPATH,xpath) 
element.clear() #input text를 초기화 한다. 

element.send_keys(location) # 주소 정보를 보낸다. 
element.send_keys(Keys.RETURN) # Enter키를 누르게 합니다.
time.sleep(1) # 로그인이 진행되는 3초간 대기


In [107]:
# 반복해서 가져올 url
category = ['프랜차이즈']#'1인분주문','프랜차이즈','치킨','피자양식','중식','한식','일식돈까스','족발보쌈','야식','분식','카페디저트','편의점'] 

# 수집된 결과가 누적될 빈 리스트
yogiyoData = []

# 카테고리별 반복 
for c in category :

    # 개별 카테고리로 이동한다. 
    driver.get('https://www.yogiyo.co.kr/mobile/#/{}'.format(c))

    while True :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # 스크롤을 가장 아래로 내린다
        time.sleep(3) # 창이 뜰때까지 기다린다 
        before_height = int(driver.execute_script("return document.body.scrollHeight")) # 현재 스크롤 위치룰 저장한다. 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # 스크롤을 다시 가장 아래로 내린다
        time.sleep(3) # 창이 뜰때까지 기다린다 
        height = int(driver.execute_script("return document.body.scrollHeight"))  # 현재 스크롤 위치를 저장한다.
        if before_height == height : # 저장된 스크롤의 높이가 지난번과 같으면 종료한다 
            break
    time.sleep(3) # 창이 뜰때까지 기다린다 
        

    # 가게 정보가 나오는 부분만 추출하기
    soup = BeautifulSoup(driver.page_source)
    storeList = soup.select('.col-sm-6') 
    storeList

    
    # 가게별 정보 가져오기 
    for s in storeList :

        # 매장명 가져오기              
        name = s.find_all("div", attrs= {'class':'restaurant-name ng-binding'})
        if name :
            store_name = name[0].text.strip()
        else :
            store_name = None 

        try : # 별점/리뷰/사장님댓글쿠폰은 없을 수 있기 때문에 예외처리로 묶음 

            # 별점 가져오기 
            star = s.find_all("span", attrs={'class': "ico-star1 ng-binding"})
            if star :
                store_star = float(star[0].text.strip().replace("★ ",""))

            # 리뷰 수 가져오기 
            review = s.find_all("span", attrs={'class': "review_num ng-binding"})
            if review[0] :
                store_review = int(review[0].text.strip().replace("리뷰 ",""))

            # 사장님 댓글 수 가져오기 
            if review[1] :
                ceo_review = int(review[1].text.strip().replace("사장님댓글 ",""))

            # 쿠폰 
            coupon = s.find_all("span", attrs={'class': "coupon-base ng-binding"})
            if coupon :
                store_coupon = coupon[0].text.strip()

        except : # 없어서 에러가 나는 경우 
            store_star = None
            store_review = 0
            ceo_review = 0
            store_coupon = '행사없음'


        # 결과 병합
        resultDic = {'카테고리':c,'매장명':store_name,"별점":store_star,'리뷰':store_review,'사장님댓글':ceo_review,'쿠폰':store_coupon}
        yogiyoData.append(resultDic)

df = DataFrame(yogiyoData)
df.set_index('매장명')
df.to_excel('요기요 (%s).xlsx' % location,sheet_name='요기요',index=False)
df


,카테고리,매장명,별점,리뷰,사장님댓글,쿠폰
0,프랜차이즈,BHC-이수점,4.8,3267,2885,"3,000원 할인"
1,프랜차이즈,버거킹-방배카페골목,NaN,0,0,행사없음
2,프랜차이즈,KFC-이수역점,NaN,0,0,행사없음
3,프랜차이즈,도미노피자-방배점,NaN,0,0,행사없음
4,프랜차이즈,엔제리너스-이수역점,4.9,648,629,"1,000원 할인"
...,...,...,...,...,...,...
407,프랜차이즈,크로플덕오리아가씨-서초점,5.0,8,8,행사없음
408,프랜차이즈,곱정당-낙성대점,NaN,0,0,행사없음
409,프랜차이즈,인생BBQ미스고기-강남2호점,NaN,0,0,행사없음
410,프랜차이즈,된장마을고깃집-서초점,NaN,0,0,행사없음


In [98]:
import pymysql
from sqlalchemy import create_engine
from pandas import DataFrame
from pandas import read_sql, read_sql_table
# 데이터베이스 접속 패키지 설치
pymysql.install_as_MySQLdb() #pip이 없음 
import MySQLdb

In [108]:
# 접속 문자열 생성
# mysql+mysqldb://계정이름:비밀번호@:포트번호/데이터베이스이름?charset=인코딩
conStr = 'mysql+mysqldb://root:1234@:3306/semiproject?charset=utf8' # utf8db 디비명 
# 데이터베이스 접속하기
engine = create_engine(conStr)
conn = engine.connect() # conn ; 접속객체

In [110]:
# 데이터 내보내기
df.to_sql(name='yogiyo',con=conn,if_exists='append',index=False) # new_table 테이블명 
conn.commit()

In [115]:
df = read_sql('SELECT * FROM yogiyo',index_col='매장명',con=conn)
df

,카테고리,별점,리뷰,사장님댓글,쿠폰
매장명,,,,,
버거킹-방배카페골목,프랜차이즈,NaN,0,0,행사없음
KFC-이수역점,프랜차이즈,NaN,0,0,행사없음
BHC-신반포점,프랜차이즈,4.8,1104,1102,"3,000원 할인"
도미노피자-방배점,프랜차이즈,NaN,0,0,행사없음
또래오래-서초방배점,프랜차이즈,NaN,0,0,행사없음
...,...,...,...,...,...
파리바게뜨-방배현대점,카페디저트,4.9,763,489,"3,000원 할인"
이삭토스트-총신대점,카페디저트,NaN,0,0,행사없음
일단시켜찹쌀타코야끼&카페,카페디저트,NaN,0,0,행사없음


In [126]:
# 데이터베이스 접속
dbcon = pymysql.connect(host='localhost',
                        port=3306,
                        user='root',
                        password='1234',
                        db = 'semiproject',
                        charset='utf8')

# 데이터 조회하기
cursor = dbcon.cursor(pymysql.cursors.DictCursor)

In [128]:
# 리뷰가 많고, 별점이 높은 순서 
sql = 'SELECT * FROM yogiyo ORDER BY 리뷰 DESC, 별점 ASC'
cursor.execute(sql) 
result1 = cursor.fetchall()
df1 = DataFrame(result1)
df1

# 데이터 내보내기
df1.to_sql(name='yogiyo_rank',con=conn,if_exists='append',index=False) # new_table 테이블명 
conn.commit()


In [130]:
# 데이터 확인하기 
df1 = read_sql('SELECT * FROM yogiyo_rank',index_col='매장명',con=conn)
df1


,카테고리,별점,리뷰,사장님댓글,쿠폰
매장명,,,,,
기대만족-본점,족발보쌈,4.9,24798,24779,행사없음
벚꽃스시-상도점,한식,4.9,6853,983,행사없음
벚꽃스시-상도점,일식돈까스,4.9,6853,983,행사없음
반올림피자-사당방배점,프랜차이즈,4.9,6578,1376,행사없음
반올림피자-사당방배점,프랜차이즈,4.9,6578,1376,"1,500원 할인"
...,...,...,...,...,...
디저트39-강남역점,카페디저트,NaN,0,0,행사없음
다이어트랩-강남점,카페디저트,NaN,0,0,행사없음
이삭토스트-총신대점,카페디저트,NaN,0,0,행사없음


In [164]:
# 리뷰가 많고, 별점이 높은 순서 
sql = 'SELECT * FROM yogiyo WHERE 쿠폰 != "행사없음" ORDER BY 쿠폰 DESC'
cursor.execute(sql) 
result1 = cursor.fetchall()
df2 = DataFrame(result1)

# 데이터 내보내기
df2.to_sql(name='yogiyo_coupon',con=conn,if_exists='append',index=False)  
conn.commit()
df2

,카테고리,매장명,별점,리뷰,사장님댓글,쿠폰
0,피자양식,피자헛-서초교대역점,4.7,808,591,"7,000원 할인"
1,프랜차이즈,피자헛-서초교대역점,4.7,808,591,"7,000원 할인"
2,피자양식,피자헛-서초교대역점,4.7,808,591,"7,000원 할인"
3,프랜차이즈,피자헛-서초교대역점,4.7,808,591,"7,000원 할인"
4,프랜차이즈,피자헛-서초교대역점,4.7,808,591,"7,000원 할인"
...,...,...,...,...,...,...
358,프랜차이즈,투썸플레이스-방배카페거리점,4.7,161,47,"1,000원 할인"
359,카페디저트,엔제리너스-이수역점,4.9,648,629,"1,000원 할인"
360,카페디저트,투썸플레이스-내방역점,4.9,299,51,"1,000원 할인"
361,프랜차이즈,엔제리너스-이수역점,4.9,648,629,"1,000원 할인"
